# Union-Find-Verfahren

## Tools für Experimente und Darstellung

Wir verwenden hier wieder `networkx` zur Repräsentation und zur Darstellung von Graphen/Bäumen. Zusätzlich verwenden wir `pygraphviz` zum besseren Ploten von Bäumen.

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout
import networkx as nx
import pygraphviz

Wir erstellen zufällig eine kleine Beispieleingabe:

In [ ]:
import itertools
import random

class Example:
    def __init__(self, no_nodes, no_union_calls):
        self.no_nodes = no_nodes
        self.union_calls = []
        
        node_pairs = list(itertools.combinations(range(no_nodes), 2))
        sampled_pairs = random.sample(node_pairs, no_union_calls)
        
        # In the sampled pairs, the node with the smaller number
        # always is the first component. We thus swap the order
        # for some of the pairs       
        for node1, node2 in sampled_pairs:
            if random.choice((True, False)):
                node1, node2 = node2, node1
            self.union_calls.append((node1, node2))

small_example = Example(10, 8)
print(small_example.union_calls)

To be able to quickly verify the results of the algorithms below, we visualize the resulting graph.

In [ ]:
graph = nx.Graph()
graph.add_nodes_from(range(small_example.no_nodes))
graph.add_edges_from(small_example.union_calls)
pos = nx.kamada_kawai_layout(graph)
nx.draw(graph, pos, with_labels=True, 
        node_size=1000, node_color='lightblue')

 ## Quick-Find

Quick-Find ist ein eher naives Verfahren, das sich für jeden Knoten eine id für die entsprechende Zusammenhangskomponente speichert. Werden zwei Zusammenhangskomponenten neu verschmolzen, muss man über alle Knoten gehen und für eine der beiden Komponenten alle id-Einträge auf den Wert der anderen Komponente setzen.

In [ ]:
class QuickFind:
    def __init__(self, no_nodes):
        self.id = list(range(no_nodes))
        self.components = no_nodes

    def find(self, v):
        return self.id[v]

    def union(self, v, w):
        id_v = self.find(v)
        id_w = self.find(w)
        if id_v == id_w:  # already in same component
            return
        # replace all occurrences of id_v in self.id with id_w
        for i in range(len(self.id)):
            if self.id[i] == id_v:
                self.id[i] = id_w
        self.components -= 1  # we merged two components
        
    def connected(self, v, w):
        return self.find(v) == self.find(w)

    def count(self):
        return self.components

In [ ]:
qf = QuickFind(small_example.no_nodes)
print("anfängliche Ids:", qf.id)
print()

for x, y in small_example.union_calls:
    qf.union(x, y)
    print("nach union(%i, %i):" % (x, y))
    print(qf.id)
    print()

In [ ]:
qf.connected(0, 6)

## Quick-Union

Quick-Union speichert alle Knoten einer Zusammenhangskomponente innerhalb eines Baums. Das erspart bei `union` das Traversieren aller Knoten, dafür kann man beim Aufruf von `find` nicht direkt eine id vergleichen, sondern vergleicht stattdessen die Wurzeln der entsprechenden Bäume.

In [ ]:
class QuickUnion:
    def __init__(self, no_nodes):
        self.parent = list(range(no_nodes))
        self.components = no_nodes

    def find(self, v):
        while self.parent[v] != v:
            v = self.parent[v]
        return v

    def union(self, v, w):
        id_v = self.find(v)
        id_w = self.find(w)
        if id_v == id_w:  # already in same component
            print("Knoten", v, "und", w, "waren bereits in",
                  "gleicher Zusammenhangskomponente.")
            return
        self.parent[id_v] = id_w
        print("Hänge Wurzel", id_v, "des Baumes von", v, 
              "an Wurzel", id_w, "des Baumes von", w)
        self.components -= 1
        
    def connected(self, v, w):
        return self.find(v) == self.find(w)

    def count(self):
        return self.components

Hier erstmal noch eine kleine Funktion zum Zeichnen des von `parent` repräsentierten Waldes:

In [ ]:
def draw_forest(parent_array):
    graph = nx.DiGraph()
    graph.add_nodes_from(range(len(parent_array)))
    for child, parent in enumerate(parent_array):
        if child != parent:
            graph.add_edge(child, parent)
        
    pos = graphviz_layout(graph.reverse(), prog='dot')
    nx.draw(graph, pos, with_labels=True, node_size=300, node_color='lightblue')

Dann testen wir einmal Quick-Union:

In [ ]:
qu = QuickUnion(small_example.no_nodes)
for x, y in small_example.union_calls:
    print("union(%i, %i)" % (x, y))
    qu.union(x, y)
    draw_forest(qu.parent)
    plt.show()
    print()

## Ranked Quick-Union

In [ ]:
class RankedQuickUnion:
    def __init__(self, no_nodes):
        self.parent = list(range(no_nodes))
        self.components = no_nodes
        self.rank = [0] * no_nodes  # [0, ..., 0]

    def find(self, v):
        while self.parent[v] != v:
            v = self.parent[v]
        return v

    def union(self, v, w):
        id_v = self.find(v)
        id_w = self.find(w)
        if id_v == id_w:
            print("Knoten", v, "und", w, "waren bereits in gleicher Zusammenhangskomponente.")
            return
        if self.rank[id_w] < self.rank[id_v]:
            self.parent[id_w] = id_v
            print("Hänge Wurzel", id_w, "des Baumes von", w, "an Wurzel", id_v, "des Baumes von", v)
        else:
            self.parent[id_v] = id_w
            print("Hänge Wurzel", id_v, "des Baumes von", v, "an Wurzel", id_w, "des Baumes von", w)

            if self.rank[id_v] == self.rank[id_w]:
                print("Update Höhe des entstandenen Baums")
                self.rank[id_w] += 1
        self.components -= 1

    def connected(self, v, w):
        return self.find(v) == self.find(w)

    def count(self):
        return self.components

In [ ]:
rqu = RankedQuickUnion(small_example.no_nodes)
for x, y in small_example.union_calls:
    print("union(%i, %i)" % (x, y))
    rqu.union(x, y)
    draw_forest(rqu.parent)
    plt.show()
    print()

## Ranked Quick-Union mit Pfadkompression

Pfadkompression ist eine Technik, die Bäume als Seiteneffekt eines `find`-Aufrufes flacher macht.

In [ ]:
class RankedQuickUnionWithPathCompression:
    def __init__(self, no_nodes):
        self.parent = list(range(no_nodes))
        self.components = no_nodes
        self.rank = [0] * no_nodes  # [0, ..., 0]

    def find(self, v):
        if self.parent[v] == v:
            return v
        root = self.find(self.parent[v])
        self.parent[v] = root
        return root

    def union(self, v, w):
        id_v = self.find(v)
        id_w = self.find(w)
        if id_v == id_w:
            return
        if self.rank[id_w] < self.rank[id_v]:
            self.parent[id_w] = id_v
        else:
            self.parent[id_v] = id_w
            if self.rank[id_v] == self.rank[id_w]:
                self.rank[id_w] += 1
        self.components -= 1

    def connected(self, v, w):
        return self.find(v) == self.find(w)

    def count(self):
        return self.components

Bezüglich `union` funktioniert alles fast genau wie ohne Pfadkompression. Allerdings ruft `union` intern `find` auf, wodurch als Seiteneffekt Knoten an die Wurzel umgehängt werden.

In [ ]:
rqupc = RankedQuickUnionWithPathCompression(small_example.no_nodes)
for x, y in small_example.union_calls:
    rqupc.union(x, y)
    
draw_forest(rqupc.parent)

Ohne Pfadkompression verändern `find`-Aufrufe die Datenstruktur nicht. Mit Pfadkompression werden die Bäume nach und nach flacher:

In [ ]:
# TODO füge für das zufällig generierte Beispiel geeignete 
# find- oder connected-Aufrufe ein
# rqupc.connected(6, 0)
# rqupc.find(2)
draw_forest(rqupc.parent)